In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.sparse as sp
from angler import Simulation
from angler.derivatives import unpack_derivs

In [2]:
from numpy import pi
from torch import device as torch_device

# Refractive indices and permittivities ================================================================================

N_si = 3.48  # silicon
eps_si = N_si ** 2

N_sio2 = 1.44  # silicon oxide
eps_sio2 = N_sio2 ** 2

N_sinitride = 1.99  # silicon nitride
eps_sinitride = N_sinitride ** 2

# Physical constants ===================================================================================================
OMEGA_1550 = 1.215e15  # 1550nm frequency
OMEGA = 2*pi*5.785e9  # 1550nm frequency
MU0 = 4 * pi * 10 ** -7  # vacuum permeability
EPSILON0 = 8.854187817620e-12  # vacuum permittivity
C = 299792458.0  # speed of light
LAMBDA = C / (OMEGA / (2 * pi))  # wavelength
k = 2*pi/LAMBDA  # wave number
# BUFFER_PERMITTIVITY = -1e20  # near infinite permittivity for cavity boundaries
BUFFER_PERMITTIVITY = 1.0  # vacuum permittivity if using PML

# Design space size ====================================================================================================
DEVICE_LENGTH = 64  # length of permittivity region
DEVICE_LENGTH_2D = 32
NPML = 4  # number of PMLs
BUFFER_LENGTH = 4  # buffer size before NPML (reflective boundary if using cavity)
SCALE = 1e-9 # 这个参数可能只是为了把omega中的e15缩放下来，不影响结果
# L0 = 1e-6 # 长度参数，Simulations库中所有的长度单位都以L0为单位
L0 = 1 # 长度参数，Simulations库中所有的长度单位都以L0为单位
dL = 0.05 # Simulations中网格间距，实际距离是dL*L0
PIXEL_SIZE = dL * L0

# Device configuration =================================================================================================

device = torch_device('cpu')


In [3]:
def maxwell_residual_2d(fields, epsilons, curl_curl_op,npml=NPML, 
                        buffer_length = BUFFER_LENGTH, buffer_permittivity = BUFFER_PERMITTIVITY,
                        add_buffer = True, trim_buffer = True):
    # nplm 是绝对吸收层的数量，电磁波传输到这里会被完全吸收
    # buffer_length 在场区的边缘添加一点点区域，然后将这里的介电系数设置为buffer_permittivity（通常取较大的值），这样可以近似认为电磁波在这里被吸收了
    '''Compute ∇×∇×E - omega^2 mu0 epsilon E'''
#     batch_size,c, W, H = fields.shape # 二维场区的宽和高，加batchsize，和通道数，构成一个四维张量
    W, H = fields.shape # 二维场区的宽和高，加batchsize，和通道数，构成一个四维张量
    
    epsilons *= torch.ones_like(fields) # 将介电系数扩展为和fields一样的形状，这样就可以直接相乘了
    
    # Add zero field amplitudes at edge points for resonator BC's
    if add_buffer:
        fields = F.pad(fields, [buffer_length+npml] * 4)
        W += 2 * (buffer_length+npml)
        H += 2 * (buffer_length+npml)
    fields = fields.view(-1, 1)
    
    # Add first layer of cavity BC's
    if add_buffer:
        epsilons = F.pad(epsilons, [buffer_length+npml] * 4, "constant", buffer_permittivity)
    epsilons = epsilons.view(-1, 1)
    
    # Compute Maxwell operator on fields
    curl_curl_E = (SCALE / L0 ** 2) * torch.matmul(curl_curl_op, fields).view( -1, 1)
    epsilon_E = (SCALE * -OMEGA ** 2 * MU0 * EPSILON0) * epsilons * fields
    
    out = curl_curl_E - epsilon_E
    out = out.view(W, H)
    
    if trim_buffer and buffer_length > 0:
        clip = buffer_length+npml
        return out[..., clip:-clip, clip:-clip] #把加入的buffer区域去掉
    else:
        return out

In [4]:
class Simulation2D:
    '''FDFD simulation of a 2-dimensional  system'''

    def __init__(self, mode = "Ez", device_length = DEVICE_LENGTH_2D, npml = 4, buffer_length = BUFFER_LENGTH,
                 buffer_permittivity = 1.0, dl = dL, l0 = L0, use_dirichlet_bcs = False):
        # device_length: 二维场区的宽和高
        # npml: 绝对吸收层的数量，电磁波传输到这里会被完全吸收
        # buffer_length: 在场区的边缘添加一点点区域，然后将这里的介电系数设置为buffer_permittivity（通常取较大的值），这样可以近似认为电磁波在这里被吸收了
        # dl: 网格间距，单位是l0
        self.mode = mode
        self.device_length = device_length
        self.npml = npml
        self.buffer_length = buffer_length
        self.buffer_permittivity = buffer_permittivity
        self.dl = dl
        self.L0 = l0
        self.use_dirichlet_bcs = use_dirichlet_bcs
    
    def get_operators(self, omega = OMEGA):
        total_length = self.device_length + 2 * self.buffer_length + 2 * self.npml
        
        perms = np.ones((total_length, total_length), dtype = np.float64)
        start = self.npml + self.buffer_length
        end = start + self.device_length
        
        # set permittivity and reflection zone
        perms[:, :start] = self.buffer_permittivity
        perms[:start, :] = self.buffer_permittivity
        perms[:, end:] = self.buffer_permittivity
        perms[end:, :] = self.buffer_permittivity
        
        sim = Simulation(omega, perms, self.dl, [self.npml, self.npml], self.mode, L0 = self.L0)
        Dyb, Dxb, Dxf, Dyf = unpack_derivs(sim.derivs)
        
        N = np.asarray(perms.shape)
        M = np.prod(N)
        
        vector_eps_z = EPSILON0 * self.L0 * perms.reshape((-1,))
        T_eps_z = sp.spdiags(vector_eps_z, 0, M, M, format = 'csr')

        curl_curl = (Dxf @ Dxb + Dyf @ Dyb)

        other = omega ** 2 * MU0 * self.L0 * T_eps_z

        return curl_curl.todense(), other.todense()

In [5]:
class MaxwellSolver2D(nn.Module):

    def __init__(self, dl,size = 32, buffer_length = 4, buffer_permittivity = 1.0, npml = 4, src_x = 16, add_buffer=True,
                 src_y = 16, channels = None, kernels = None, drop_p = 0.1):

        super().__init__()

        self.size = size
        self.src_x = src_x
        self.src_y = src_y
        self.npml = npml
        self.use_complex = (self.npml > 0)
        self.add_buffer = add_buffer
        self.buffer_length = buffer_length
        self.buffer_permittivity = buffer_permittivity
        self.drop_p = drop_p
        self.dl = dl
        self.sim = Simulation2D(device_length = self.size, buffer_length = self.buffer_length, npml = self.npml,
                                buffer_permittivity = self.buffer_permittivity,dl = self.dl)
        curl_curl_op, _ = self.sim.get_operators()
        self.curl_curl_op = torch.tensor(np.asarray(np.real(curl_curl_op)), device = device).float()
    def get_Mawell_loss(self, fields, epsilons, trim_buffer = True):
        # Compute Maxwell operator on fields
        residuals = maxwell_residual_2d(fields, epsilons, self.curl_curl_op,
                                        buffer_length = self.buffer_length, add_buffer = self.add_buffer, trim_buffer = trim_buffer)
        return residuals

In [ ]:
aa = torch.tensor([1,2,3])
print(aa[1:-1])

In [6]:
x = np.linspace(0,0.24,448)
y = np.linspace(0,0.24,448)
xf,yf = np.meshgrid(x,y)
dx = x[1]-x[0]
dy = y[1]-y[0]
R = np.abs(xf+yf)/np.sqrt(2)

E = torch.tensor(np.real(np.exp(-1j*k*R)),dtype=torch.float32)

dE_x = torch.diff(E[1:-1,:],2,1)/dx**2
dE_y = torch.diff(E[:,1:-1],2,0)/dy**2

maxwell_loss = dE_x+dE_y+k**2*E[1:-1,1:-1]
print(E.shape)

torch.Size([448, 448])


In [7]:
EE = E.view(-1,64,64)
print(EE[0].shape)

torch.Size([64, 64])


In [12]:
test = MaxwellSolver2D(size=EE.shape[-1],dl=dx)
test_loss = test.get_Mawell_loss(EE[0],1.0)
for i in range(1,EE.shape[0]):
    test_loss += test.get_Mawell_loss(EE[i],1.0)

print(torch.mean(test_loss))

tensor(6.4687e-06)
